# Trabajo Práctico 1: Análisis Exploratorio de Datos
## Fecha y hora de entrega máxima:
11/04/2022 18:00

## Acerca del Dataset "Datos de clientes del banco"
Los datos están relacionados con campañas de marketing directo (llamadas telefónicas) de una institución bancaria portuguesa. El objetivo de la clasificación es predecir si el cliente suscribirá un depósito a plazo.

## 1. Listado de variables y selección
- Por cada variable explicar en una oración el contenido de dicha variable y definir si será utilizada como variable de entrada, de salida, o no será utilizada.
- Para la variable de salida especificar los valores posibles que puede tener.
- Por cada variable que no se vaya a utilizar, explicar brevemente el motivo.

<br />

|Variable|Descripción|Entrada|Salida|Sin utilización|
|--|---|---|--|--|
|age|Edad del cliente|X  |||
|job|Tipo de trabajo del cliente|X  |||
|marital|Estado civil del cliente|X  |||
|education|Tipo de educación del cliente|X  |||
|default|Indica si el cliente tiene crédito en mora|X  |||
|balance|Balance actual del cliente|X  |||
|housing|Indica si el cliente tiene un prestamo de vivienda|X  |||
|loan|Indica si el cliente tiene un prestamo personal|X  |||
|contact|Tipo de comunicación con el cliente|X  |||
|day|Día de la semana en que hubo contacto con el cliente por última vez |X  |||
|month|Mes del año en que hubo contacto con el cliente por últtima vez|X  |||
|duration|Duración en segundos del último contacto con el cliente|X  |||
|campaign|Número de contactos realizados al cliente durante la campaña|X  |||
|pdays|Número de días transcurridos desde la última vez que se contacto al cliente desde una campaña anterior|X  |||
|previous|Número de contactos realizados al cliente antes de la campaña actual|X  |||
|poutcome|Resultado de la campaña de marketing anterior|X  |||
|term_deposit|Indica si el cliente ha suscrito un depósito a término||X||

<br />

En cuanto a la variable de salida "term_deposit", los valores posibles que puede tener son un string binario "yes" o "no". No posee otro valor que éstos dos, con lo cual los datos de la variable de salida son utilizables sin necesidad de limpiarlos.

<br />

En cuanto a las variables que no se utilizarán, no elegimos ninguna, ya que consideramos que TODAS las variables de entrada posiblemente tengan una correlación con la variable de salida "term_deposit".

In [ ]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import keras
import tensorflow as tf
import h5py
import PIL
import sklearn_pandas
import plotly.express as px

In [ ]:
# If we import the dataset from the csv file we see we have values with the value "unknown"
dataset_original = pd.read_csv("BankCustomerData.csv")

# We copy the dataset so we don't change directly the original dataset after working on it
ds = dataset_original

# Return the first five rows of the DataFrame
ds.head()

In [ ]:
# To replace these values with NaN, we must provide a list with all missing value formats
missing_value_formats = ["unknown", "n.a.","?","NA","n/a", "na", "--"]
ds = pd.read_csv("BankCustomerData.csv", na_values = missing_value_formats)

# Display the firsts and lasts lines of the file
ds

In [ ]:
# Get the length of the file
len(ds)

In [ ]:
# See dataset dimensions
ds.shape

In [ ]:
# Show statistical data
ds.describe()

In [ ]:
# Return the columns structure of the DataFrame
ds.dtypes

In [ ]:
# Return the NULL quantity values in each column
ds.isnull().sum()

## 2. Análisis detallado de un conjunto de variables

### 1. Para la variable de salida, explicar y graficar su balanceo y qué consecuencias va a tener eso a la hora de entrenar y medir el rendimiento de distintos modelos.

In [ ]:
# See percentage of yes and no in term_deposit as Bar Plot
ax = ds.term_deposit.value_counts().plot.bar(figsize=(5,5))

ax.set_ylabel('Cantidad')
ax.legend()
ax.set_title('Cantidad de clientes que se suscribieron o no a un depósito a plazo')

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1, p.get_height() * 1))

In [ ]:
# Show the distribution of the output variable also in plotly!
px.histogram(ds, x='term_deposit', title = 'Cantidad de clientes que se suscribieron o no a un depósito a plazo')

In [ ]:
# See percentage of yes and no in term_deposit as Pie Plot
ds.term_deposit.value_counts().plot.pie(autopct='%1.0f%%', figsize=(5,5)).set_title('Cantidad de usuarios suscriptos o no a un depósito a plazo')

In [ ]:
# See percentage of yes and no in term_deposit as Pie Plot also in Plotly!
fig = px.pie(
             values=ds.term_deposit.value_counts(), 
             names=['Yes', 'No'], 
             title='Cantidad de usuarios suscriptos o no a un depósito a plazo',
             color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [ ]:
yes_counts = ds[ds.term_deposit == "yes"].term_deposit.count()
no_counts = ds[ds.term_deposit == "no"].term_deposit.count()
# In the case of total_counts, it does not work with .value_counts(), so instead use len()
total_counts = len(ds)

data = [[yes_counts, "{0: 5.2f} %".format((yes_counts / total_counts) * 100)],
[no_counts, "{0: 5.2f} %".format((no_counts / total_counts) * 100)],
[total_counts, 100]]

col_headers=["Quantity", "Percentage"]
row_headers=["yes", "no", "Total"]

print(pd.DataFrame(data, row_headers, col_headers))

Como se puede observar en el gráfico de barras y en el de torta, existe un **claro desequilibrio** en los valores de la variable de salida "term_deposit", ya que el **90.5%** de los clientes decidió no suscribirse a un depósito a plazo, mientras que un **9.5%** sí lo hizo.

Debido a la gran cantidad de valores 'no', se podría llegar a afectar el entrenamiento y la medición del rendimiento del modelo, ya que podría aprender a predecir que en la mayoría de los casos, los clientes no se suscribirán a un depósito a plazo para evitar fallas.

Cabe destacar que la variable de salida no posee valores NaN, lo cual es bueno para la precisión del modelo.

### 2. Para 5 variables de entrada (elegidas o no, pero incluyendo al menos 3 elegidas) graficar y explicar cómo se comportan y cómo afectan a la variable de salida.

Para las variables de entrada, elegimos las siguientes 5 columnas:
1. age
2. duration
3. job
4. education
5. housing y loan


#### 1. Variable Age:

In [ ]:
# Show the distribution of the age variable
px.histogram(ds, x='age', title = 'Age Distribution')

In [ ]:
# Show the relation of the output variable with the current variable being analized
px.histogram(ds, x='age', color='term_deposit', barmode='group')

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "mean"})

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "max"})

In [ ]:
ds.groupby("term_deposit").aggregate({"age": "min"})

En cuanto a la edad del cliente en relación a la variable term_deposit, podemos observar considerando el histograma, que esta variable de entrada guarda relación con la variable de salida term_deposit debido a que  se observa una tendencia alcista hasta la edad de 32 años, para luego ir decreciendo a lo largo del gráfico. 

Esto significa que el lograr que un cliente decida suscribirse o no a un depósito a plazo dependerá en gran medida de la edad de la persona.


Ésto podría suceder a raíz de que las personas que poseen cerca de 30 años poseen una capacidad de ahorro mayor, con lo cual la gran mayoría de las campañas para que un cliente decida suscribirse o no a un depósito a plazo están dirigidas hacia personas con una franza de edad de entre 30 y 50 años aproximadamente, con un promedio de edad de 40 años tanto para los "yes" como para los "no"

#### 2. Variable Duration

In [ ]:
# Show the distribution of the output variable
px.histogram(ds, x='duration', title = 'duration Distribution')

In [ ]:
# Show the relation of the output variable with the current variable being analized
px.histogram(ds, x='duration', color='term_deposit', barmode='group')

In [ ]:
duration_no_pago_a_termino = ds[ds.term_deposit == "no"].duration.sum() / len(ds[ds.term_deposit == "no"].term_deposit)
duration_pago_a_termino = ds[ds.term_deposit == "yes"].duration.sum() / len(ds[ds.term_deposit == "yes"].term_deposit)

print('Duracion promedio de no pago a termino: ', duration_no_pago_a_termino)
print('Duracion promedio de pago a termino: ',  duration_pago_a_termino)

In [ ]:
ds.groupby("term_deposit").aggregate({"duration": "min"})

In [ ]:
ds[ds.duration == 0]

In [ ]:
ds.groupby("term_deposit").aggregate({"duration": "max"})

Los valores dicen que cuando se suscriben a un plazo a término, el contacto dura más del doble que cuando no se suscriben.

Otra cosa que nos llamó la atención, es ver que como valores mínimos la variable duration presenta 0 minutos. Ésto quiere decir, que al llamar le cortaron inmediatamente, por lo que claramente, term_deposit será igual a cero.
Es por ésto, que éste atributo afecta en gran medida a la variable de salida ya que, si la duración=0 entonces term_deposit='no' en todos los casos.

Sin embargo, la duración no se conoce antes de realizar una llamada. Además, una vez finalizada la llamada, obviamente se conoce term_deposit. 

Por lo tanto, esta entrada sólo debería incluirse a efectos de evaluación comparativa y debería descartarse si la intención es tener un <b>MODELO DE PREDICCIÓN REALISTA.</b>

#### 3. Variable Job

In [ ]:
px.histogram(ds, x='job', color='term_deposit', barmode='group')

In [ ]:
f,ax=plt.subplots(1,3,figsize=(16,6))

ax[0].set_title('Numero de veces que aparece cada valor')
valores_distintos = ds.job.value_counts()
valores_distintos.sort_values().plot.bar(ax=ax[0])

ax[1].set_title('% de veces que NO se suscribio a un DP')
no_pago_a_termino = ds[ds.term_deposit == 'no'].job.value_counts() / ds.job.value_counts()
no_pago_a_termino.sort_values().plot.bar(ax=ax[1])

ax[2].set_title('% de veces que se suscribio a un DP')
pago_a_termino = ds[ds.term_deposit == 'yes'].job.value_counts() / ds.job.value_counts()
pago_a_termino.sort_values().plot.bar(ax=ax[2])

De la variable Job podemos observar segun el histograma que al igual del resto de las varibles presenta un gran desbalanceo. Mirando los datos vemos que hay valores que se presentan mayor cantidad de veces en la muestra, lo cual puede hacer sobre entrenar al modelo o restarle importancia algun valor que no aparece tanto.

En los porcentajes, el valor 'student' es el que mas veces se suscribe a un depósito a plazo, seguido por 'retired' y 'unemployed'. Los primeros dos tienen sentido, ya que tanto un estudiante como una persona retirada tienen, en teoria, dinero ahorrado ya que no tienen responsabilidades laborales. En el caso del desempleado, también tiene sentido viendolo por ese lado, ya que está buscando trabajo y debe cuidar los ahorros generados para que no se deprecien.

Por el lado de los que no se suscriben a un depósito a plazo, aparece 'blue-collar', 'entrepreneur', 'housemaid', y 'services'. Trabajos de diferente índole que tal vez por tener demasiados gastos no tienen capacidad de ahorro, por lo cual necesitan el dinero para el dí a día, y por eso no se suscriben.

Otro análisis importante es que la categoría 'blue-collar', la cual es la primera en cuanto a cantidad de apariciones, es la úlitma en cuanto a porcentaje de veces que paga a término.
Sin embargo, la segunda categoría que aparece en cuanto a cantidad de apariciones llamada 'management', es la cuarta en porcentaje de veces que paga a término.

#### 4. Variable Education:

In [ ]:
sns.countplot(x='education',hue='term_deposit',data=ds)
plt.xlabel('education')
plt.title('Distribucion de education por term_deposit')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols= 3, figsize=(16,6))

ds_cleaned = ds[ds['education'].notna()]

for c, i in zip(np.sort(ds_cleaned.education.unique().astype('str')), range(3)):
    ds_cleaned[ds_cleaned.education == c].term_deposit.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=axes[i])
    axes[i].set_title(c)

Se puede observar una tendencia alsista en el porcentaje de clientes que se suscriben a un plazo fijo cuando subimos el nivel de educación.
Es decir, los clientes que tiene mayores niveles de educación, tienden a tener un porcentaje de suscripción más alto que los que poseen bajo nivel de educación.

Por tal motivo podemos decir que 'education' se trata de una variable importante para decidir si un cliente hará una suscripción .

#### 5. Variable Housing y Loan:

In [ ]:
plt.figure(figsize=[15,15])

plt.subplot(221)
sns.countplot(x='housing',hue='term_deposit',data=ds)
plt.xlabel('housing')
plt.title('Distribucion de housing por term_deposit')

plt.subplot(222)
sns.countplot(x='loan',hue='term_deposit',data=ds)
plt.xlabel('loan')
plt.title('Distribucion de loan por term_deposit')

Analizando los gráficos de housing y loan, podemos deducir que tener un préstamo de vivienda o personal no afecta, por si mismo, si un cliente se suscribirá o no.

Por lo que en principio, ambas variables no se categorizarían como una variable importante para predecir la variable target.

### 3. Para cada una de las variables de entrada elegidas, explicar si se debería realizar o no alguna transformación para poder utilizarla como entrada de un modelo y por qué.


<b>Método: OneHotEncoder

Variables afectadas: job, marital, education, default, housing, loan, contact, month, poutcome, term_deposit</b>

Justificación: Permitirá transformar las variables categóricas mencionadas en numéricas, agregando una nueva columna por valor de categoría de cada variable.
    
<b>Método: Scalling
 
Variables afectadas: balance, duration</b>
    
Justificación: Ambas variables numéricas se encuentran en rangos muy diferentes al resto, es por esto que las escalaremos a rangos similares sin alterar su distribución. Esto puede ayudar a algunos modelos a converger más rápido y evitar que sufran por valores que dominen las operaciones por estar en rangos más grandes.

### 4. Analizar si las variables de entrada seleccionadas presentan valores nulos y/o extremos. En caso de encontrar dichas condiciones, indicar qué tratamiento se podría darle a las mismas y por qué.


#### 1. Chequear cantidad de valores nulos:

In [ ]:
# Check the quantity of NULLs in each column 
ds.isnull().sum()

In [ ]:
# Check the quantity of NOT NULLs values in each column 
ds.info()

Como podemos ver, solo 4 columnas (job, education, contact and poutcome) presentan valores NaN.
Si comparamos en porcentaje cuánta cantidad en dichas columnas representan los valores NULL tenemos:

In [ ]:
index = ['job', 'education', 'contact', 'poutcome']
df = pd.DataFrame({'NaN values': ds.isnull().sum(),
                   'All values': ds.count()}, index=index)
ax = df.plot.bar(rot=0, figsize=(20,10))

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1, p.get_height() * 1))

In [ ]:
nulls_job = ds.job.isnull().sum()
not_null_jobs = ds.job.count()
total_jobs = nulls_job + not_null_jobs

nulls_education = ds.education.isnull().sum()
not_null_education = ds.education.count()
total_education = nulls_education + not_null_education

nulls_contact = ds.contact.isnull().sum()
not_null_contact = ds.contact.count()
total_contact = nulls_contact + not_null_contact

nulls_poutcome = ds.poutcome.isnull().sum()
not_null_poutcome = ds.poutcome.count()
total_poutcome = nulls_poutcome + not_null_poutcome

data = [[nulls_job, not_null_jobs, total_jobs, "{0: 5.2f} %".format(((nulls_job / total_jobs)*100))],
[nulls_education, not_null_education, total_education, "{0: 5.2f} %".format(((nulls_education / total_education)*100))],
[nulls_contact, not_null_contact, total_contact, "{0: 5.2f} %".format(((nulls_contact / total_contact)*100))],
[nulls_poutcome, not_null_poutcome, total_poutcome, "{0: 5.2f} %".format(((nulls_poutcome / total_poutcome)*100))]]

col_headers=["NaN Quantity", "Not NaN Quantity", "Total Quantity", "Percentage of NaN"]
row_headers=["job", "education", "contact", "poutcome"]

print(pd.DataFrame(data, row_headers, col_headers))

#### 2. Chequear outliers:


In [ ]:
fig = px.box(ds, y="age")
fig.show()

In [ ]:
ds.age.duplicated().value_counts()


Podemos visualizar la variable "age" aplicando un diagrama de caja, observando que el 50 por ciento de nuestra distribución (percentiles 25 a 75) se concentra entre los 33 y 48 años, el valor mínimo el cual también coincide con el valor más pequeño que cae dentro del rango de datos incluidos es de 18 años, y el valor más alto que se ubica dentro del rango de puntos es 70, teniendo así un máximo de 95 años. 
**Como outliers podríamos tomar a las edades por encima de 70 años, teniendo outliers ligeros como 71 años, y outliers extremos como 95 años.**

Como el resto de variables de entrada son strings, no podemos aplicarles el diagrama de caja sin antes aplicarles algún método para poder expresarlas en números. El análisis actual solo se limita a analizar variables numéricas.

No será necesario eliminar columnas duplicadas porque más allá de que se podría repetir age (de hecho se repiten en 42564 filas), duration, job, education, housing o loan, no es aplicable ya que un mismo cliente podría tener la misma edad, el mismo trabajo, o la misma educación.  

Como conclusión tenemos que la variable age presenta outliers, y la variable poutcome presenta valores NaN en un porcentaje del 85%.

### 5. Verificar si existen variables altamente correlacionadas con la variable "target". En dicho caso, explicar por qué considera que esto pasa.
(ver duration)
Emi

In [ ]:
ds["term_deposit"] = ds.term_deposit.replace(['no','yes'], [0, 1])
ds.corr()

In [ ]:
ds["term_deposit"] = ds.term_deposit.replace(['no','yes'], [0, 1])
sns.heatmap(ds.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(10, 10)
plt.show()

Como observamos en mapa de calor tenemos las siguientes correlaciones positivas/negativas entre la variable target term_deposit y algunas variables de entrada:

## 3. Hipótesis sobre los datos

### 1. Formular hipótesis sobre la variable target bajo determinadas condiciones
*Hipótesis 1:* Si el cliente tiene un préstamo personal y de vivienda (loan y housing), el cliente no está interesado en suscribirse a un depósito a plazo.

*Hipótesis 2:* Relacionando la edad del cliente y su edad (age y job) puede ser que tome o no una suscripción a un depósito a plazo dependiendo las siguientes combinaciones:
- 18 - 25 años y es 'student'
- 25 - 60 años y es 'management'
- 20 - 60 años y es 'blue-collar'
Mauro

*Hipótesis 3:* Si el cliente está casado, y trabaja de 'blue-collar' es probable que no saque un depósito a plazo 
Emi

*Hipótesis 4:* Si el cliente está casado, y trabaja de 'management' es probable que no saque un depósito a plazo 
Emi

*Hipótesis 4:* Si el cliente tiene educacion primaria y trabaja de 'blue-collar' (education y job) es probable que no saque un depósito a plazo
Mauro

### 2. Comprobación de hipotesis


#### Hipótesis 1: 
Si el cliente tiene un préstamo personal y de vivienda (loan y housing), el cliente no está interesado en suscribirse a un depósito a plazo.

In [ ]:
ds_loan_housing = ds[(ds['loan']=='yes')&(ds['housing']=='yes')]

ds_loan_housing.term_deposit.value_counts().plot.pie(autopct='%1.0f%%', figsize=(8,8))

Para verificar esta hipótesis, se confeccionó un gráfico de torta que muestre para todos los clientes que tienen cŕédito personal y crédito de vivienda a la vez, información que indique si se suscribe a un plazo fijo o no.

A partir de la gráfica obtenida, podemos concluir que la hipotesis es verdadera, y existe una tendencia del cliente a no suscribirse a un plazo fijo cuando posee credito personal y de vivienda a la vez.

#### Hipótesis 2: 
Relacionando la edad del cliente y su edad (age y job) puede ser que tome o no una suscripción a un depósito a plazo dependiendo las siguientes combinaciones:
- 18 - 25 años y es 'student'
- 25 - 60 años y es 'management'
- 20 - 60 años y es 'blue-collar'